<a href="https://colab.research.google.com/github/GaoangLiu/AA_ipynb/blob/master/Bag_of_Words_Meets_Bags_of_Popcorn_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!curl -o train.tsv ali.140714.xyz:8000/popcorn.tsv
!curl -o test.tsv ali.140714.xyz:8000/popcorn_test.tsv
!curl -o master.csv ali.140714.xyz:8000/imdb_master.csv
!curl -o sample.csv ali.140714.xyz:8000/popcorn_sample.csv

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 32.0M  100 32.0M    0     0  3427k      0  0:00:09  0:00:09 --:--:-- 4120k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 31.2M  100 31.2M    0     0  3271k      0  0:00:09  0:00:09 --:--:-- 3919k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  129M  100  129M    0     0  3779k      0  0:00:35  0:00:35 --:--:-- 4094k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  276k  100  276k    0     0   171k      0  0:00:01  0:00:01 --:--:--  171k


In [26]:
extra = pd.read_csv('master.csv', encoding="latin-1")
extra.drop(extra[extra.label=='unsup'].index, inplace=True)
extra['sentiment'] = (extra['label'] == 'pos').astype(int)
extra= extra[['review', 'sentiment']]
# extra.label.value_counts(), extra
extra

train = pd.read_csv('train.tsv', sep='\t')
train = train[['review', 'sentiment']]
train = pd.concat([extra, train])
train.sentiment.value_counts()

1    37500
0    37500
Name: sentiment, dtype: int64

In [4]:
import tensorflow as tf 
import pandas as pd 
import numpy as np 
import sklearn.metrics
import seaborn as sns

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [27]:
import os 
import numpy as np
import pandas as pd
import seaborn as sb
import tensorflow as tf
import tensorflow_hub as hub
import sklearn
import sklearn.metrics 
import sklearn.model_selection
import sklearn.feature_extraction
from absl import logging
import gensim.downloader as api
logging.set_verbosity(logging.INFO)

import re
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

class OverwriteLog():
    # overwrite logging for kaggle kernel
    def info(self, msg):
        print(msg)

stop_words = set(stopwords.words("english")) 
lemmatizer = WordNetLemmatizer()


def clean_text(text):
    text = re.sub(r'[^\w\s]','',text, re.UNICODE)
    text = text.lower()
    text = [lemmatizer.lemmatize(token, 'v') for token in text.split(" ")]
    text = [word for word in text if not word in stop_words]
    text = " ".join(text)
    return text

def load_data():
    train = pd.read_csv('train.tsv', sep='\t')
    test = pd.read_csv('test.tsv', sep='\t')

    extra = pd.read_csv('master.csv', encoding="latin-1")
    extra.drop(extra[extra.label=='unsup'].index, inplace=True)
    extra['sentiment'] = (extra['label'] == 'pos').astype(int)
    extra= extra[['review', 'sentiment']]

    train = train[['review', 'sentiment']]
    train = pd.concat([extra, train])

    train['text'] = train.review
    train['target'] = train.sentiment
    test['text'] = test.review
    return train, test
    

def countvectorize(train, test, vocab_size, sentence_len):
    tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=vocab_size)
    text = pd.concat([train.text, test.text])
    tokenizer.fit_on_texts(text)

    sequence_train = tokenizer.texts_to_sequences(train.text)
    train_inputs = tf.keras.preprocessing.sequence.pad_sequences(sequence_train, maxlen=sentence_len)
    logging.info('Train text tokeninzed')

    sequence_test = tokenizer.texts_to_sequences(test.text)
    test_inputs = tf.keras.preprocessing.sequence.pad_sequences(sequence_test, maxlen=sentence_len)
    logging.info('Test text tokeninzed')

    return train_inputs, test_inputs, tokenizer

def embed_word_vector(word_index, vocab_size=10000, embed_size=128, model='glove-wiki-gigaword-100'):
    glove = api.load(model) # default: wikipedia 6B tokens, uncased
    zeros = [0] * embed_size
    matrix = np.zeros((vocab_size, embed_size))
      
    for word, i in word_index.items(): 
        if i >= vocab_size or word not in glove: continue # matrix[0] is zeros, that's also why >= is here
        matrix[i] = glove[word]

    logging.info('Matrix with embedded word vector created')
    return matrix

def build_model(vocab_size = 10000, max_len = 100, embed_size = 128, embed_matrix=[]):
    text_input = tf.keras.Input(shape=(max_len, ))
    embed_text = tf.keras.layers.Embedding(vocab_size, embed_size)(text_input)
    if len(embed_matrix) > 0:
        embed_text = tf.keras.layers.Embedding(vocab_size, embed_size, \
                                        weights=[embed_matrix], trainable=False)(text_input)
        
    net = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(128, return_sequences=True))(embed_text)
    net = tf.keras.layers.GlobalMaxPool1D()(net)

    net = tf.keras.layers.Dense(64, activation='relu')(net)
    net = tf.keras.layers.Dropout(0.1)(net)

    net = tf.keras.layers.Dense(32, activation='relu')(net)
    net = tf.keras.layers.Dropout(0.1)(net)
    outputs = tf.keras.layers.Dense(1, activation='sigmoid')(net)
    
    model = tf.keras.models.Model(inputs=text_input, outputs=outputs)

    return model

"""split the following codes into several chunks in Jyputer 
for clearer reading and saved variables
"""

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


'split the following codes into several chunks in Jyputer \nfor clearer reading and saved variables\n'

In [0]:
vocab_size = 10000
sentence_len = 150
embed_size = 300

train, test = load_data()
train_inputs, test_inputs, tokenizer = countvectorize(train, test, vocab_size, sentence_len)
X_train, X_val, y_train, y_val = sklearn.model_selection.train_test_split(train_inputs, train.target, test_size=0.2, random_state=0)
matrix = embed_word_vector(tokenizer.word_index, vocab_size, embed_size, "word2vec-google-news-300")
# matrix = embed_word_vector(tokenizer.word_index, vocab_size, embed_size, "fasttext-wiki-news-subwords-300")
logging.info("Data loaded and split")


INFO:absl:Train text tokeninzed
INFO:absl:Test text tokeninzed


[===-----------------------------------------------] 7.3% 121.7/1662.8MB downloaded

In [0]:
# train_inputs.shape
# tokenizer = tf.keras.preprocessing.text.Tokenizer()
# text = pd.concat([train.text, test.text])
# tokenizer.fit_on_texts(text)
# tokenizer.num_words

In [0]:
# len(tokenizer.word_index)

In [137]:
# Build model 
model = build_model(vocab_size, sentence_len, embed_size, matrix)
model.summary()
model.compile(loss=tf.keras.losses.BinaryCrossentropy(), optimizer='adam', metrics=['accuracy'])
logging.info("Model built")



INFO:absl:Model built


Model: "model_13"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_16 (InputLayer)        [(None, 150)]             0         
_________________________________________________________________
embedding_21 (Embedding)     (None, 150, 300)          3000000   
_________________________________________________________________
bidirectional_13 (Bidirectio (None, 150, 256)          439296    
_________________________________________________________________
global_max_pooling1d_13 (Glo (None, 256)               0         
_________________________________________________________________
dense_33 (Dense)             (None, 64)                16448     
_________________________________________________________________
dropout_20 (Dropout)         (None, 64)                0         
_________________________________________________________________
dense_34 (Dense)             (None, 32)                208

In [138]:
# Run model
checkpoint = tf.keras.callbacks.ModelCheckpoint('best.h5', monitor='val_accuracy', save_best_only=True, verbose=1)
earlystopping = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=5, verbose=1)

train_history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=30,
    callbacks=[checkpoint, earlystopping],
    batch_size=1024,
    verbose=1
)
logging.info("Model trainning complete")


Epoch 1/30
20/20 [==============================] - ETA: 0s - loss: 0.6459 - accuracy: 0.6524
Epoch 00001: val_accuracy improved from -inf to 0.77920, saving model to best.h5
20/20 [==============================] - 9s 464ms/step - loss: 0.6459 - accuracy: 0.6524 - val_loss: 0.5094 - val_accuracy: 0.7792
Epoch 2/30
20/20 [==============================] - ETA: 0s - loss: 0.4693 - accuracy: 0.7879
Epoch 00002: val_accuracy improved from 0.77920 to 0.82800, saving model to best.h5
20/20 [==============================] - 8s 420ms/step - loss: 0.4693 - accuracy: 0.7879 - val_loss: 0.3969 - val_accuracy: 0.8280
Epoch 3/30
20/20 [==============================] - ETA: 0s - loss: 0.4004 - accuracy: 0.8206
Epoch 00003: val_accuracy improved from 0.82800 to 0.84680, saving model to best.h5
20/20 [==============================] - 8s 421ms/step - loss: 0.4004 - accuracy: 0.8206 - val_loss: 0.3555 - val_accuracy: 0.8468
Epoch 4/30
20/20 [==============================] - ETA: 0s - loss: 0.3613 -

INFO:absl:Model trainning complete


Epoch 00018: early stopping


In [139]:
# validation & predict
model.load_weights('best.h5')
y_preds = model.predict(test_inputs).round().astype(int)
logging.info("Prediction DONE")


sub = pd.read_csv('sample.csv')
sub['sentiment'] = y_preds
sub.to_csv('popcorn_submission.csv', index=False)!curl -X PUT --upload-file popcorn_submission.csv ali.140714.xyz:8000/


INFO:absl:Prediction DONE


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0  227k    0     0    0     0      0      0 --:--:--  0:00:01 --:--:--     0

## Received: "popcorn_submission.csv"

100  227k    0    41  100  227k     25   142k  0:00:01  0:00:01 --:--:--  142k
